# Import

In [ ]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
from file_py.run_log_parser import RunLogParser
from file_py.csv_preprocessing_scaler_full import CsvPreprocessingScalerFull

# Preprocessing

In [ ]:
df = CsvPreprocessingScalerFull.read_csv_file("file_csv/LogSplunk_12_06_Full.csv")

In [ ]:
df_raw = CsvPreprocessingScalerFull.RawPreprocessing(df)

df_Le = CsvPreprocessingScalerFull.LEPreprocessing(df)
df_OH = CsvPreprocessingScalerFull.OhePreprocessing(df)

In [ ]:
df_std_LE = CsvPreprocessingScalerFull.stdScaler(CsvPreprocessingScalerFull.LEPreprocessing(df))
df_std_OH = CsvPreprocessingScalerFull.stdScaler(CsvPreprocessingScalerFull.OhePreprocessing(df))

# Test

In [ ]:
file_path = 'file_csv/attackLog_12_06.csv'
result_df_Le = RunLogParser.process_attacks(file_path, CsvPreprocessingScalerFull.stdScaler(CsvPreprocessingScalerFull.LEPreprocessing(df)))
result_df_OH = RunLogParser.process_attacks(file_path, CsvPreprocessingScalerFull.stdScaler(CsvPreprocessingScalerFull.OhePreprocessing(df)))

# Graphic Analysis of Attack

In [ ]:
from file_py.plots import Plots

In [ ]:
result_df_Raw = RunLogParser.process_attacks(file_path, CsvPreprocessingScalerFull.RawPreprocessing(df))

In [ ]:
Plots.plot_cake_attack(result_df_Raw)

In [ ]:
Plots.plot_top_10_mitre_id(result_df_Raw)

Qui si può notare come generalmente le __regole scattate più volte__ sono anche quelle che hanno effettivamente __risposto a più attachi__.  
Tra le regole presenti nella prima tabella solo __T1003.002__ e __T1548__ NON si trovano nella seconda.

In [ ]:
Plots.plot_value_counts_per_unique(result_df_Raw)

Grazie a questo grafico invece possiamo giungere ad una serie di conclusioni.  
Su __66__ regole diverse:  
- quelle scattate in risposta ad ALMENO un attacco reale sono __44__.  
Le __22__ regole che si sono attivate senza rispondere ad attacchi sono:  
T1555.004, T1552.004, T1546.011, T1082, T1021.003, T1218, T1566.001, T1218.010, T1204.002, T1218.011, T1053.002, T1552.006, T1059.007, T1070.001, T1562.002, T1562.001, T1136.001, T1133, T1059.005, T1615, T1560.001, T1197

- delle 44 in risposta ad attacchi reali:  
    - __20__ si sono attivate più volte per __non-attacchi__ rispetto che per gli attacchi
    - __8__ si sono attivate lo __stesso numero__ di volte per attacchi e non-attacchi
    - __16__ si sono attivate più volte in risposta ad __attacchi__ rispetto che a non-attacchi

In [ ]:
Plots.plot_distributions(result_df_Raw)

In questo grafico invece notiamo che la distribuzione tra attacchi e non-attacchi:
- secondo il "__severity_id__" rimane __stabile__;
- secondo il "__tag__" rimane stabile ad eccezione di "__attack communicate malware network__" che in rapporto agli altri registra un numero considerevole di non-attacchi in confronto agli attacchi;
- secondo l' "__EventType__" rimane stabile ad eccezione del "__4__" che in rapporto agli altri registra un numero considerevole di non-attacchi in confronto agli attacchi

# Correlation Matrix

In [ ]:
from file_py.correlation_matrix_plots import CorrelationMatrixPlots

In [ ]:
CorrelationMatrixPlots.plot_correlation_matrix(result_df_Le, 'Correlation Matrix (Label Encoding)')
CorrelationMatrixPlots.plot_correlation_matrix_big(result_df_OH, 'Correlation Matrix (OneHot Encoding)')

# ML

In [ ]:
from file_py.preprocessing_train_test_split import PreprocessingTrainTestSplit
from file_py.initial_training import InitialTraining
from file_py.hyperparameter_tuning import HyperparameterTuning
from file_py.advanced_models import AdvancedModels
from file_py.deep_learning_model import DeepLearningModel
from file_py.model_evaluator import ModelEvaluator

### OneHot

In [ ]:
# Split data
X_train_OH, X_test_OH, y_train_OH, y_test_OH = PreprocessingTrainTestSplit.split_data(result_df_OH, "corrisponde_ad_attacco")

# Initial model training and evaluation
InitialTraining.train_and_evaluate_initial_models(X_train_OH, y_train_OH, X_test_OH, y_test_OH)

# Hyperparameter tuning
best_models_OH = HyperparameterTuning.tune_hyperparameters(X_train_OH, y_train_OH)

# Evaluate best models on test set
evaluator_OH = ModelEvaluator(best_models_OH)
evaluation_results_OH = evaluator_OH.evaluate_models(X_test_OH, y_test_OH)

# Train XGBoost model
AdvancedModels.train_xgboost(X_train_OH, y_train_OH, X_test_OH, y_test_OH)

# Train deep learning model
DeepLearningModel.train_deep_learning_model(X_train_OH, y_train_OH, X_test_OH, y_test_OH)

### Label

In [ ]:
# Split data
X_train_Le, X_test_Le, y_train_Le, y_test_Le = PreprocessingTrainTestSplit.split_data(result_df_Le, "corrisponde_ad_attacco")

# Initial model training and evaluation
InitialTraining.train_and_evaluate_initial_models(X_train_Le, y_train_Le, X_test_Le, y_test_Le)

# Hyperparameter tuning
best_models_Le = HyperparameterTuning.tune_hyperparameters(X_train_Le, y_train_Le)

# Evaluate best models on test set
evaluator_Le = ModelEvaluator(best_models_Le)
evaluation_results_Le = evaluator_Le.evaluate_models(X_test_Le, y_test_Le)

# Train XGBoost model
AdvancedModels.train_xgboost(X_train_Le, y_train_Le, X_test_Le, y_test_Le)

# Train deep learning model
DeepLearningModel.train_deep_learning_model(X_train_Le, y_train_Le, X_test_Le, y_test_Le)

Qui notiamo che il __modello__ che ci restituisce il _risultato migliore_ con i dati attuali sia dopo il OneHot che dopo il Label è il __KNN__ con un'accuratezza rispettivamente di 0.90/0.91 e 0.91/0.93